In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import h5py
import glob

In [2]:
matplotlib.rc('xtick', labelsize=14)
matplotlib.rc('ytick', labelsize=14)
matplotlib.rcParams['font.size']=14

In [3]:
def calculate_fourier_conversion(Boxsize):
    '''
    Takes the Boxsize in kpc and produces the conversion factors
    from Fourier units to physical units. For use with genPK output.
    '''
    Boxsize = Boxsize / 1000 # convert to Mpc

    k_conv = 2*np.pi / Boxsize
    p_conv = (Boxsize / (2 * np.pi))**3
    
    return k_conv, p_conv

In [4]:
def get_genPK_data(fpath, k_conv, p_conv):
    
    genPK = np.loadtxt(fpath)
    
    bins = genPK[:, 0]
    pk = genPK[:, 1]
    
    dk = pk * (2 * np.pi)**3 * (4 * np.pi) * bins**3
    
    bins = genPK[:, 0] * k_conv
    
    pk = genPK[:, 1] * p_conv
    
    return bins, pk, dk

In [5]:
def wnumber2wlength(k):
    return 1/k
def wlength2wnumber(l):
    return 1/l

In [6]:
def load_data(pk_path, snap_path):
    
    snap = h5py.File(snap_path, 'r')
    
    redshift = np.round(float(snap['Header'].attrs['Redshift']), decimals=1)
    boxsize = np.round(float(snap['Header'].attrs['BoxSize']), decimals=1)
    
    k_conv, p_conv = calculate_fourier_conversion(boxsize)
    
    bins, pk, dk = get_genPK_data(pk_path, k_conv, p_conv)
    
    return np.array([bins, pk, dk]), redshift, boxsize

In [7]:
# separates data by redshift
def sort_data(pk_paths, snap_paths):
    
    datas = []
    redshifts = []
    boxsizes = []
    
    for p_path, s_path in zip(pk_paths, snap_paths):
    
        data, redshift, boxsize = load_data(p_path, s_path)
        
        datas.append(data)
        redshifts.append(redshift)
        boxsizes.append(boxsize)
    
    datas = np.array(datas)
    redshifts = np.array(redshifts)
    boxsizes = np.array(boxsizes)
    
    # doing this to remove duplicates; prevents looping too many times
    zs = list(set(redshifts))
    
    s = []

    
    for redshift in zs:
        
        truth = ( redshift == redshifts )
        
        s.append( [datas[truth], redshifts[truth], boxsizes[truth]] )
    
    
    '''
    reference for s:
    
    s contains many indices because it is a list of arrays of arrays of arrays...
    
    the first index, s[i], changes which redshift the data corresponds to
    e.g., s[0] will give all data corresponding to z=8.96
    
    the second index, s[i][j], selects what info you're looking at:
    s[i][0] is the arrays of power spectrum data
    s[i][1] is the array of redshifts
    s[i][2] is the array of boxsizes
    
    the third index, s[i][j][k], selects which particular dataset you're looking at
    e.g., s[0][0][1] will give the first 2cDM power spectrum data at z=8.96
    
    by convention, the s[i][j][0] will correspond to the CDM simulation
    
    this itself, s[i][0][k], will contain 3 arrays:
    
    s[i][j][0] = [k_bins, raw_pk, dk]
    
    this will be a mouthful to process, but should make automating plotting easier
    
    ''' 
    
    return s

In [17]:
def make_plots(cdm_pk_paths, sidm_pk_paths, cdm_snap_paths, sidm_snap_paths, out_path, labeler=''):
    
    cdm_s = sort_data(cdm_pk_paths, cdm_snap_paths)
    sidm_s = sort_data(sidm_pk_paths, sidm_snap_paths)
    
    # this for each loop will pull out different datsets per redshift
    for i in range(len(sidm_s)):
        
        redshift = cdm_s[i][1][0] # safe to assume this since they should be all the same redshift
                              # could add a check later
        
        cdm_pk_data = cdm_s[i][0] # 0th element is the data for all snapshots
        
        sidm_pk_data = sidm_s[i][0]
        
        cdm_bins = cdm_pk_data[:, 0]
        cdm_pks = cdm_pk_data[:, 1]
        cdm_dks = cdm_pk_data[:, 2]
        
        sidm_bins = sidm_pk_data[:, 0]
        sidm_pks = sidm_pk_data[:, 1]
        sidm_dks = sidm_pk_data[:, 2]
        
        # raw plots
        
        fig, ax = plt.subplots(figsize=[12,12])
        
        for j in range(1, len(cdm_bins)):
            ax.plot(cdm_bins[j], cdm_pks[j], linestyle='--')
        
        for j in range(1, len(sidm_bins)):
            ax.plot(sidm_bins[j], sidm_pks[j])
        
        ax.set_xlabel('k (h/Mpc)')
        ax.set_ylabel('$P(k)$')
        ax.set_xscale('log')
        ax.set_yscale('log')
        
        ax.plot([],[], label='CDM', linestyle='--', color='k')
        ax.plot([],[], label='2cDM', color='k')
        ax.plot([],[], label='z = {}'.format(redshift), alpha=0)
        ax.plot([],[], label='(0, 0) $\sigma = 0.1$', alpha=0)
        ax.plot([],[], label='$\Delta m/m$ = 5.56e-8', alpha=0)
        ax.set_xlim(0, 10**3)

        sax = ax.secondary_xaxis('top', functions=(wnumber2wlength, wlength2wnumber))
        sax.set_xlabel('$\lambda$ (Mpc/h)')
        plt.title('{} Power Spectrum z = {}'.format(labeler, redshift))
        plt.legend()
        plt.grid(True, which="both", ls="-")
        fname = '{}_Power_Spectrum_z={}.png'.format(labeler, redshift)
        plt.savefig(out_path+fname, dpi=150)
        #plt.show()
        plt.close()
        
        
        # dk plots
        
        fig, ax = plt.subplots(figsize=[12,12])
        
        for j in range(1, len(cdm_bins)):
            ax.plot(cdm_bins[j], cdm_dks[j], linestyle='--')
        
        for j in range(1, len(sidm_bins)):
            ax.plot(sidm_bins[j], sidm_dks[j])
        
        ax.set_xlabel('k (h/Mpc)')
        ax.set_ylabel('$\Delta^{2}(k)$')
        ax.set_xscale('log')
        ax.set_yscale('log')

        ax.plot([],[], label='CDM', linestyle='--', color='k')
        ax.plot([],[], label='2cDM', color='k')
        ax.plot([],[], label='z = {}'.format(redshift), alpha=0)
        ax.plot([],[], label='(0, 0) $\sigma = 0.1$', alpha=0)
        ax.plot([],[], label='$\Delta m/m$ = 5.56e-8', alpha=0)
        ax.set_xlim(0, 10**3)

        sax = ax.secondary_xaxis('top', functions=(wnumber2wlength, wlength2wnumber))
        sax.set_xlabel('$\lambda$ (Mpc/h)')
        plt.title('{} Dimensionless Power Spectrum z = {}'.format(labeler, redshift))
        plt.legend()
        plt.grid(True, which="both", ls="-")
        fname = '{}_Dimless_Spectrum_z={}.png'.format(labeler, redshift)
        plt.savefig(out_path+fname, dpi=150)
        #plt.show()
        plt.close()
        
        
        # spectrum ratio plots
        
        fig, ax = plt.subplots(figsize=[12,12])
        
        ratios = np.divide(sidm_pks, cdm_pks)
        
        for j in range(1, len(sidm_bins)):
            ax.plot(sidm_bins[j], ratios[j])
        
        ax.set_xlabel('k (h/Mpc)')
        ax.set_ylabel('$P_{2cDM}/P_{CDM}$')
        ax.set_xscale('log')
        #ax.set_yscale('log')

        ax.plot([],[], label='z = {}'.format(redshift), alpha=0)
        ax.plot([],[], label='(0, 0) $\sigma = 0.1$', alpha=0)
        ax.plot([],[], label='$\Delta m/m$ = 5.56e-8', alpha=0)
        ax.set_xlim(0, 10**3)
        ax.set_ylim(-0.1, 1.5)

        sax = ax.secondary_xaxis('top', functions=(wnumber2wlength, wlength2wnumber))
        sax.set_xlabel('$\lambda$ (Mpc/h)')
        plt.title('Ratio of DM Power Spectra z = {}'.format(redshift))
        plt.legend()
        plt.grid(True, which="both", ls="-")
        fname = '{}_Spectrum_Ratio_z={}.png'.format(labeler, redshift)
        plt.savefig(out_path+fname, dpi=150)
        #plt.show()
        plt.close()
 
        ############
        ##AVERAGES##
        ############
        
        avg_cdm_pk = np.mean(cdm_pks, axis=0)
        std_cdm_pk = np.std(cdm_pks, axis=0)
        avg_cdm_dk = np.mean(cdm_dks, axis=0)
        std_cdm_dk = np.std(cdm_dks, axis=0)
        
        
        avg_sidm_pk = np.mean(sidm_pks, axis=0)
        std_sidm_pk = np.std(sidm_pks, axis=0)
        avg_sidm_dk = np.mean(sidm_dks, axis=0)
        std_sidm_dk = np.std(sidm_dks, axis=0)
        
        avg_ratio = np.mean(ratios, axis=0)
        std_ratio = np.std(ratios, axis=0)
        
        #avg_ratio = np.divide(avg_sidm_pk, avg_cdm_pk)
        #std_ratio = np.sqrt( np.power(avg_cdm_pk, -2)*np.square(std_sidm_pk)+\
        #                    np.square(avg_sidm_pk * std_cdm_pk)*np.power(avg_cdm_pk, -4))
        
        # raw plots
        
        fig, ax = plt.subplots(figsize=[12,12])
        
        ax.plot(cdm_bins[0], avg_cdm_pk, label='Average CDM', color='k', linestyle='--')
        ax.fill_between(cdm_bins[0], avg_cdm_pk - std_cdm_pk, avg_cdm_pk + std_cdm_pk, color='k', alpha=0.3, hatch="\\\\")
        
        ax.plot(sidm_bins[0], avg_sidm_pk, label='Average 2cDM', color='r')
        ax.fill_between(sidm_bins[0], avg_sidm_pk - std_sidm_pk, avg_sidm_pk + std_sidm_pk, color='b', alpha=0.3, hatch='//')
        
        ax.set_xlabel('k (h/Mpc)')
        ax.set_ylabel('$P(k)$')
        ax.set_xscale('log')
        ax.set_yscale('log')

        ax.plot([],[], label='z = {}'.format(redshift), alpha=0)
        ax.plot([],[], label='(0, 0) $\sigma = 0.1$', alpha=0)
        ax.plot([],[], label='$\Delta m/m$ = 5.56e-8', alpha=0)
        ax.set_xlim(0, 10**3)

        sax = ax.secondary_xaxis('top', functions=(wnumber2wlength, wlength2wnumber))
        sax.set_xlabel('$\lambda$ (Mpc/h)')
        plt.title('{} Power Spectrum z = {}'.format(labeler, redshift))
        plt.legend()
        plt.grid(True, which="both", ls="-")
        fname = '{}_AVG_Power_Spectrum_z={}.png'.format(labeler, redshift)
        plt.savefig(out_path+fname, dpi=150)
        #plt.show()
        plt.close()
        
        
        # dk plots
        
        fig, ax = plt.subplots(figsize=[12,12])
        
        ax.plot(cdm_bins[0], avg_cdm_dk, label='Average CDM', color='k', linestyle='--')
        ax.fill_between(cdm_bins[0], avg_cdm_dk - std_cdm_dk, avg_cdm_dk + std_cdm_dk, color='k', alpha=0.3, hatch='\\\\')
        
        ax.plot(sidm_bins[0], avg_sidm_dk, label='Average 2cDM', color='r')
        ax.fill_between(sidm_bins[0], avg_sidm_dk - std_sidm_dk, avg_sidm_dk + std_sidm_dk, color='b', alpha=0.3, hatch='//')
        
        ax.set_xlabel('k (h/Mpc)')
        ax.set_ylabel('$\Delta^{2}(k)$')
        ax.set_xscale('log')
        ax.set_yscale('log')

        ax.plot([],[], label='z = {}'.format(redshift), alpha=0)
        ax.plot([],[], label='(0, 0) $\sigma = 0.1$', alpha=0)
        ax.plot([],[], label='$\Delta m/m$ = 5.56e-8', alpha=0)
        ax.set_xlim(0, 10**3)

        sax = ax.secondary_xaxis('top', functions=(wnumber2wlength, wlength2wnumber))
        sax.set_xlabel('$\lambda$ (Mpc/h)')
        plt.title('{} Dimensionless Power Spectrum z = {}'.format(labeler, redshift))
        plt.legend()
        plt.grid(True, which="both", ls="-")
        fname = '{}_AVG_Dimless_Spectrum_z={}.png'.format(labeler, redshift)
        plt.savefig(out_path+fname, dpi=150)
        #plt.show()
        plt.close()
        
        

        # spectrum ratio plots
        
        fig, ax = plt.subplots(figsize=[12,12])
        
        ax.plot(sidm_bins[0], avg_ratio, label='Average 2cDM')
        
        ax.fill_between(sidm_bins[0], avg_ratio - std_ratio, avg_ratio + std_ratio, alpha=0.3, color='k')
        
        ax.set_xlabel('k (h/Mpc)')
        ax.set_ylabel('$P_{2cDM}/P_{CDM}$')
        ax.set_xscale('log')
        #ax.set_yscale('log')

        ax.plot([],[], label='z = {}'.format(redshift), alpha=0)
        ax.plot([],[], label='(0, 0) $\sigma = 0.1$', alpha=0)
        ax.plot([],[], label='$\Delta m/m$ = 5.56e-8', alpha=0)
        ax.set_xlim(0, 10**3)
        ax.set_ylim(-0.1, 1.5)

        sax = ax.secondary_xaxis('top', functions=(wnumber2wlength, wlength2wnumber))
        sax.set_xlabel('$\lambda$ (Mpc/h)')
        plt.title('Ratio of {} Power Spectra z = {}'.format(labeler, redshift))
        plt.legend()
        plt.grid(True, which="both", ls="-")
        fname = '{}_AVG_Spectrum_Ratio_z={}.png'.format(labeler, redshift)
        plt.savefig(out_path+fname, dpi=150)
        #plt.show()
        plt.close()
        

In [18]:
l1 = ['/media/ryan/Elements/run_2cDM_L3N128/snap_00{}.hdf5'.format(i) for i in range(1,8)]
l2 = ['/media/ryan/Elements/run_2cDM_L3N128_var2/snap_00{}.hdf5'.format(i) for i in range(1,8)]
l3 = ['/media/ryan/Elements/run_2cDM_L3N128_var3/snap_00{}.hdf5'.format(i) for i in range(1,8)]
l4 = ['/media/ryan/Elements/run_2cDM_L3N128_var4/snap_00{}.hdf5'.format(i) for i in range(1,8)]
l5 = ['/media/ryan/Elements/run_2cDM_L3N128_var5/snap_00{}.hdf5'.format(i) for i in range(1,8)]
l6 = ['/media/ryan/Elements/run_2cDM_L3N128_var6/snap_00{}.hdf5'.format(i) for i in range(1,8)]

sidm_snap_paths = l1 + l2 + l3 + l4 + l5 + l6

In [19]:
q1 = ['/media/ryan/Elements/run_CDM_L3N128/snap_00{}.hdf5'.format(i) for i in range(1,8)]
q2 = ['/media/ryan/Elements/run_CDM_L3N128_var2/snap_00{}.hdf5'.format(i) for i in range(1,8)]
q3 = ['/media/ryan/Elements/run_CDM_L3N128_var3/snap_00{}.hdf5'.format(i) for i in range(1,8)]
q4 = ['/media/ryan/Elements/run_CDM_L3N128_var4/snap_00{}.hdf5'.format(i) for i in range(1,8)]
q5 = ['/media/ryan/Elements/run_CDM_L3N128_var5/snap_00{}.hdf5'.format(i) for i in range(1,8)]
q6 = ['/media/ryan/Elements/run_CDM_L3N128_var6/snap_00{}.hdf5'.format(i) for i in range(1,8)]

cdm_snap_paths = q1 + q2 + q3 + q4 + q5 + q6

In [20]:
cdm_str = 'PK-DM-snap_00{}_CDM.hdf5'


c1 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128/'+cdm_str.format(i) for i in range(1,8)]
c2 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var2/'+cdm_str.format(i) for i in range(1,8)]
c3 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var3/'+cdm_str.format(i) for i in range(1,8)]
c4 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var4/'+cdm_str.format(i) for i in range(1,8)]
c5 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var5/'+cdm_str.format(i) for i in range(1,8)]
c6 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var6/'+cdm_str.format(i) for i in range(1,8)]

cdm_pk_paths = c1 + c2 + c3 + c4 + c5 + c6

In [21]:
tcdm_str = 'PK-DM-snap_00{}_2cDM.hdf5'


d1 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128/'+tcdm_str.format(i) for i in range(1,8)]
d2 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var2/'+tcdm_str.format(i) for i in range(1,8)]
d3 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var3/'+tcdm_str.format(i) for i in range(1,8)]
d4 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var4/'+tcdm_str.format(i) for i in range(1,8)]
d5 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var5/'+tcdm_str.format(i) for i in range(1,8)]
d6 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var6/'+tcdm_str.format(i) for i in range(1,8)]

sidm_pk_paths = d1 + d2 + d3 + d4 + d5 + d6

In [22]:
#cdm_pk_paths, sidm_pk_paths, cdm_snap_paths, sidm_snap_paths, out_path
outpath = '/home/ryan/Downloads/plots_L3N128_var/'
make_plots(cdm_pk_paths, sidm_pk_paths, cdm_snap_paths, sidm_snap_paths, outpath, labeler='DM')

/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero

/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:191: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:221: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by ze

/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:107: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:160: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by ze

In [23]:
tcdm_str = 'PK-by-snap_00{}_2cDM.hdf5'

b1 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128/'+tcdm_str.format(i) for i in range(1,8)]
b2 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var2/'+tcdm_str.format(i) for i in range(1,8)]
b3 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var3/'+tcdm_str.format(i) for i in range(1,8)]
b4 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var4/'+tcdm_str.format(i) for i in range(1,8)]
b5 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var5/'+tcdm_str.format(i) for i in range(1,8)]
b6 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var6/'+tcdm_str.format(i) for i in range(1,8)]

sidm_pk_paths = b1 + b2 + b3 + b4 + b5 + b6

In [24]:
cdm_str = 'PK-by-snap_00{}_CDM.hdf5'

x1 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128/'+cdm_str.format(i) for i in range(1,8)]
x2 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var2/'+cdm_str.format(i) for i in range(1,8)]
x3 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var3/'+cdm_str.format(i) for i in range(1,8)]
x4 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var4/'+cdm_str.format(i) for i in range(1,8)]
x5 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var5/'+cdm_str.format(i) for i in range(1,8)]
x6 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var6/'+cdm_str.format(i) for i in range(1,8)]

cdm_pk_paths = x1 + x2 + x3 + x4 + x5 + x6

In [25]:
#cdm_pk_paths, sidm_pk_paths, cdm_snap_paths, sidm_snap_paths, out_path
outpath = '/home/ryan/Downloads/plots_L3N128_var/'
make_plots(cdm_pk_paths, sidm_pk_paths, cdm_snap_paths, sidm_snap_paths, outpath, labeler='BY')

/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero

/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:191: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:221: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by ze

/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:107: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:160: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by ze